In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict 
import sys
import os
import matplotlib.pyplot as plt
from tabulate import tabulate

In [48]:
ADDR = '/nfs_home/nbhardwaj/data/set_data/400_'
ORIG_ADDR = '/nfs_home/SPEC_2006/400.perlbench/400.perlbench.pinatrace.csv'
final_ADDR = '/nfs_home/nbhardwaj/data/rds_data/400/'

### Testing integrity of split files

In [25]:
sets = [x for x in range(64)]
f = open(ADDR+'test.csv', "a+")
for curr_set in sets:
    df = pd.read_csv(ADDR + str(curr_set)+'.csv')
    for x in df.ICount.values:
        f.write(str(x)+'\n')
f.close()

In [28]:
print(len(df.ind.unique()))

354799939


In [41]:
df = pd.read_csv(ORIG_ADDR, skiprows = [0], usecols = ['ICount'], dtype = {'ICount':'str'})

In [52]:
df.head()

,ICount
0,638310950169
1,638310950170
2,638310950172
3,638310950175
4,638310950177


In [53]:
df.tail()

,ICount
356213642,639310950158
356213643,639310950159
356213644,639310950161
356213645,639310950162
356213646,639310950166


In [49]:
print(len(df.ICount.unique()))

354799938


### Calculating Reuse Distance

In [49]:
sets = [x for x in range(64)]
for cset in sets:
    df = pd.read_csv(ADDR + str(cset)+'.csv')
    m = defaultdict(list)
    # Map : <Data val> : <indexes where it appears>
    for ind, x in enumerate(df.Data.values):
        m[x].append(ind)
    rd_map = {}
    for k in m.keys():
        rds = np.diff(np.asarray(m[k]), append = sys.maxsize)
        rds = np.where(rds>64, 65, rds)
        rd_map.update(zip(m[k], rds))
    df['rd'] = df.index.to_series().map(rd_map)
    df.Data = df.Data.astype('int64')
    df['delta'] = np.diff(df.Data.values, prepend = np.nan)
    df.to_csv(ADDR + str(cset)+'.csv')

In [59]:
# len(df)

# df.head()

# df.tail()

# m = defaultdict(list)
# # Map : <Data val> : <indexes where it appears>
# for ind, x in enumerate(df.Data.values):
#     m[x].append(ind)

# rd_map = {}
# for k in m.keys():
#     rds = np.diff(np.asarray(m[k]), append = sys.maxsize)
#     rds = np.where(rds>64, 65, rds)
#     rd_map.update(zip(m[k], rds))

# df['rd'] = df.index.to_series().map(rd_map)

# df.Data = df.Data.astype('int64')

# df['delta'] = np.diff(df.Data.values, prepend = np.nan)

# df



266937